In [ ]:
from glob import glob
import pandas as pd
from io import StringIO
import numpy as np
from collections import namedtuple
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
a = int("11010010",2)
b = int("11010101",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

a = int("11010010",2) - int("1101",2)
b = int("11010101",2) - int("1101",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

bin(int("11010101",2) - 
int("10101010",2))

a = int("11010010",2) - int("101011",2)
b = int("11010101",2) - int("101011",2)
diff = b - a

bin(a - diff), bin(b + 2*diff)

In [ ]:
num =  int("11010010",2)
pred = int("11010101",2)
center = 4
# shift_calculation(num, 32 - center, pred, domain=4)

### Shift and LZC calculation

In [ ]:
def shift_calculation(num, center, pred, domain=4, bits=32):
    string_repr = np.binary_repr(pred,bits)
    right,left = min(center + domain, bits), max(center - domain, 0)
    last_bit = string_repr[left-1]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*(domain+(32-right))
    tmp = string_repr[:left]
    h = tmp+goal
    shift = int(goal,2) - pred
    return shift

In [ ]:
def lzc(val, bits=32):
    """Count leading zeroes."""
    cnt = 0
    for i in range(0, bits):
        if val & (1 << (bits - 1 - i)) != 0:
            break
        cnt += 1
    return cnt
lzcu = np.frompyfunc(lzc, 2, 1)

# Test
## Read data and prepare dataframe

In [ ]:
truth = np.fromfile("test.npf", "int64")

In [ ]:
df = pd.DataFrame({"truth":truth})
df["prediction"] = 0

df["shifted_prediction"] = 0
df["shifted_truth"] = 0
df["shift"] = 0

df["shifted_residual"] = 0
df["normal_residual"] = 0

df["shifted_lzc"] = 0
df["normal_lzc"] = 0

df.head()

## Calculate predictions (shifted and normal)

In [ ]:
for i in range(2,df.truth.size):
    df["prediction"][i] = df["truth"][i-1]
    center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1])
    df["shift"][i] = shift_calculation(None, center,df["prediction"][i],domain=4)
    df["shifted_prediction"][i] = df["prediction"][i] + df["shift"][i]
    df["shifted_truth"][i] = df["truth"][i] + df["shift"][i]
    df["shifted_residual"][i] = df["shifted_prediction"][i] ^ df["shifted_truth"][i]
    df["normal_residual"][i] = df["prediction"][i] ^ df["truth"][i]
    df["shifted_lzc"][i] = lzc(df["shifted_prediction"][i] ^ df["shifted_truth"][i])
    df["normal_lzc"][i] = lzc(df["prediction"][i] ^ df["truth"][i])
df.head()

## Analysis

In [ ]:
analysis = [ 
   (df["normal_lzc"] <= df["shifted_lzc"]).sum(), 
   (df["shifted_lzc"]-df["normal_lzc"]).sum(), 
   df["normal_lzc"].sum(), 
   df["shifted_lzc"].sum(),
   df["normal_lzc"].sum()/(df["normal_lzc"].size*32) * 100,
   df["shifted_lzc"].sum()/(df["shifted_lzc"].size*32) * 100
]
print(
    """
    Shifted LZC >= Former LZC (of 1000): {0}
    Sum Shifted LZC vs. Sum Former LZC : {2} vs. {3} (diff: {1})
    % Shifted LZC vs. % Former LZC : {4}% vs. {5}%
    """.format(*analysis))

## Calculate reconstructions (shifted and normal)

In [ ]:
# Reconstruction of original true value

In [ ]:
df["reconstruct"] = 0
df["shifted_reconstruct"] = 0

In [ ]:
for i in range(2,df.truth.size):
    center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1])
    shift = shift_calculation(None, center,df["prediction"][i],domain=4)
    shifted_truth = (df["prediction"][i] + shift) ^ (df["shifted_residual"][i] + shift) - shift
    df["shifted_reconstruct"][i] = shifted_truth - shift
    df["reconstruct"][i] = df["prediction"][i] ^ df["normal_residual"][i]

In [ ]:
(np.array_equal(df["truth"][2:],df["reconstruct"][2:]),
np.array_equal(df["truth"][2:],df["shifted_reconstruct"][2:]))
# reconstruct successful?

In [ ]:
df.tail()

## Compare with previous error addition

In [ ]:
df = df[["truth","prediction"]]
df.head()

In [ ]:
df["prev_error_prediction"] = 0

In [ ]:
delta = 0
beta, parts = 1,1
for i in range(1, df["prev_error_prediction"].size):
    delta = df["truth"][i-1] - df["prediction"][i-1]
    overshot = delta < 0
    offset = np.abs(delta)
    correction = (beta * offset) / parts
    if not overshot:
        df["prev_error_prediction"][i] =  df["prediction"][i] + correction
    elif correction <= df["prediction"][i]:
        df["prev_error_prediction"][i] = df["prediction"][i] - correction
    else:
        df["prev_error_prediction"][i] = 0
        offset = 0

In [ ]:
df["lzc_normal"] = lzcu(df['prediction'] ^ df['truth'], 32)
df["lzc_prev_error"] = lzcu(df['prev_error_prediction'] ^ df['truth'], 32)

In [ ]:
df.head()

In [ ]:
df["lzc_normal"].mean(),df["lzc_prev_error"].mean()

In [ ]:
df[["lzc_normal","lzc_prev_error"]].max(axis=1).mean()

# Improve upon the shifted reconstruction

In [ ]:
def shift_calculation(num, center, pred, domain=4, bits=32):
    """
    Shifted residue calculation
    
    
    
    """
    string_repr = np.binary_repr(pred,bits)
    right,left = min(center + domain, bits), max(center - domain, 0)
    last_bit = string_repr[left-1]
    tmp = string_repr[:left]
    if last_bit == "0":
        val = "10"
    else:
        val = "01"
    goal = val*((32-left)//2)
    h = tmp+goal
    if len(h) == 31:
        h = h + h[-2]
    shift = int(goal,2) - pred
    return shift

In [ ]:
truth = np.fromfile("test.npf", "int64")
def shifted_residual_calculation(truth, do, centeroffset):

    df = pd.DataFrame({"truth":truth})
    df["prediction"] = 0

    df["shifted_prediction"] = 0
    df["shifted_truth"] = 0
    df["shift"] = 0

    df["shifted_residual"] = 0
    df["normal_residual"] = 0

    df["shifted_lzc"] = 0
    df["normal_lzc"] = 0

    for i in range(2,df.truth.size):
        df["prediction"][i] = df["truth"][i-1]
        center = 32 - lzc(df["truth"][i-2] ^ df["truth"][i-1]) + centeroffset
        df["shift"][i] = shift_calculation(None, center,df["prediction"][i],domain=do)
        df["shifted_prediction"][i] = df["prediction"][i] + df["shift"][i]
        df["shifted_truth"][i] = df["truth"][i] + df["shift"][i]
        df["shifted_residual"][i] = df["shifted_prediction"][i] ^ df["shifted_truth"][i]
        df["normal_residual"][i] = df["prediction"][i] ^ df["truth"][i]
        df["shifted_lzc"][i] = lzc(df["shifted_prediction"][i] ^ df["shifted_truth"][i])
        df["normal_lzc"][i] = lzc(df["prediction"][i] ^ df["truth"][i])
    analysis = [ 
       (df["normal_lzc"] <= df["shifted_lzc"]).sum(), 
       (df["shifted_lzc"]-df["normal_lzc"]).sum(), 
       df["normal_lzc"].sum(), 
       df["shifted_lzc"].sum(),
       df["normal_lzc"].sum()/(df["normal_lzc"].size*32) * 100,
       df["shifted_lzc"].sum()/(df["shifted_lzc"].size*32) * 100
    ]
    print("Potential:", df[["normal_lzc","shifted_lzc"]].max(axis=1).sum())
    print(
        """
        Shifted LZC >= Former LZC (of 1000): {0}
        Sum Shifted LZC vs. Sum Former LZC : {2} vs. {3} (diff: {1})
        % Shifted LZC vs. % Former LZC : {4}% vs. {5}%
        """.format(*analysis))

In [ ]:
for do in range(1,11):
    for centeroffset in range(-4,5):
        print(do, centeroffset)
        shifted_residual_calculation(truth, do=do, centeroffset=centeroffset)

In [ ]:
binarize = np.frompyfunc(np.binary_repr, 2 , 1)

In [ ]:
# binarize(df[["truth","prediction","shifted_prediction","shifted_truth","shifted_xor"]], 32).head()

In [ ]:
df["shifted_xor"] = df["shifted_prediction"] ^ df["shifted_truth"]